In [1]:
from Rag_chat import Chat

In [3]:
ch = Chat()

/home/mahdi/loan_recommender_chatbot/information_retrival.py:13: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceEmbeddings(model_name="hiieu/halong_embedding")
/home/mahdi/loan_recommender_chatbot/myvenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
ch.QA_with_rag("سقف وام بهان ")

/home/mahdi/loan_recommender_chatbot/Rag_chat.py:35: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = self.llm(prompt)


'سقف وام بهان تا ۱۰۰ میلیون تومان است.'

In [25]:
import pandas as pd
import re

In [2]:
# logic.py
"""
Python port of your loan suggestion logic from logic.js.
Functions:
  - update_with_la(records, la)
  - update_with_da(records, da)
  - query_complex(scenarios, deposit_amount=None, repayment_duration=None,
                  deposit_duration=None, interest_rate=None, credit_score=None)
  - calculate_sort_order(loan)

Expect each record to be a dict with fields matching your JS schema.
Requires a JSON file 'parameters_weights.json' in the same directory.
"""
import json
from typing import List, Dict, Any, Optional

# Load parameter weights once
def _load_weights() -> Dict[str, Any]:
    with open('parameters_weights.json', 'r', encoding='utf-8') as f:
        return json.load(f)

_parameters_weights = _load_weights()


In [3]:
import json
from typing import List, Dict, Any, Optional

# Load parameter weights once
def _load_record() -> Dict[str, Any]:
    with open('MEC-LoanRecomn_Scenarios-V15.json', 'r', encoding='utf-8') as f:
        return json.load(f)

_records = _load_record()


In [4]:
_records

[{'id': 1,
  'nickname': 'بهان(بدون ضامن)',
  'package_name': 'فرابانک',
  'contract_type': 'مرابحه',
  'granted_method': 'واریز به حساب',
  'loan_amount_limit': 1000000000,
  'deposit_duration': 3,
  'interest_rate': 23,
  'repayment_duration': 60,
  'loan_coefficient': 1000,
  'credit_score': 'A',
  'minimum_deposit_amount': 'nan',
  'maximum_deposit_amount': 'nan',
  'minimum_loan_amount': 100000000,
  'guarantee': 'خیر',
  'receiving_channel': 'غیر حضوری'},
 {'id': 2,
  'nickname': 'بهان(بدون ضامن)',
  'package_name': 'فرابانک',
  'contract_type': 'مرابحه',
  'granted_method': 'واریز به حساب',
  'loan_amount_limit': 1000000000,
  'deposit_duration': 3,
  'interest_rate': 23,
  'repayment_duration': 60,
  'loan_coefficient': 1000,
  'credit_score': 'B',
  'minimum_deposit_amount': 'nan',
  'maximum_deposit_amount': 'nan',
  'minimum_loan_amount': 100000000,
  'guarantee': 'خیر',
  'receiving_channel': 'غیر حضوری'},
 {'id': 3,
  'nickname': 'شایان',
  'package_name': 'شایان یک',
  'c

In [5]:
def calculate_sort_order(loan: Dict[str, Any]) -> None:
    """
    Mutates loan by adding a 'sortOrder' key based on weighted criteria,
    faithfully mirroring the JS logic.
    """
    # Determine bucket
    loanAmountKey = 'out_of_range'
    la = loan.get('loan_amount', 0)
    if la <= 500_000_000:
        loanAmountKey = '1-50'
    elif la <= 1_000_000_000:
        loanAmountKey = '50-100'
    elif la <= 1_500_000_000:
        loanAmountKey = '100-150'
    elif la <= 2_000_000_000:
        loanAmountKey = '150-200'
    elif la <= 2_500_000_000:
        loanAmountKey = '200-250'
    elif la <= 3_000_000_000:
        loanAmountKey = '250-300'

    pw = _parameters_weights
    # Extract individual weights
    ir_key = str(loan.get('interest_rate', ''))
    rd_key = str(loan.get('repayment_duration', ''))
    dd_key = str(loan.get('deposit_duration', ''))
    # CS: first A-E or N
    cs_match = re.search(r'[ABCDE]', loan.get('credit_score', '') or '')
    cs_key = cs_match.group(0) if cs_match else 'N'

    ir_value = pw['IR'][loanAmountKey].get(ir_key, 0)
    rd_value = pw['RD'][loanAmountKey].get(rd_key, 0)
    w_type_coef = pw['w_type'][loanAmountKey].get(loan.get('nickname', ''), 1)
    dd_value = pw['DD'].get(dd_key, 0)
    cs_value = pw['CS'].get(cs_key, 0)

    # Global weights
    w = pw['w']
    coef = (
        ir_value * w['IR_score'] +
        rd_value * w['RD_score'] +
        dd_value * w['DD_score'] +
        cs_value * w['CS_score']
    )
    # Business weight
    w_business = pw['w_business'].get(loan.get('nickname', ''), 1)

    # Final score
    # loan['w_IR_score'] = w['IR_score']
    # loan['ir_value'] = ir_value
    # loan['w_RD_score'] = w['RD_score']
    # loan['rd_value'] = rd_value
    # loan['w_DD_score'] = w['DD_score']
    # loan['dd_value'] = dd_value
    # loan['w_CS_score'] = w['CS_score']
    # loan['cs_value'] = cs_value
    
    # loan['w_type_coef'] = w_type_coef
    # loan['coef'] = coef
    # loan['w_business'] = w_business
    loan['sortOrder'] = coef * w_type_coef * w_business


In [6]:

def update_with_la(records: List[Dict[str, Any]], la: float) -> List[Dict[str, Any]]:
    """
    Given desired loan amount 'la', update each record's
    loan_amount, monthly_repayment, deposit_amount,
    then filter and return valid records.
    """
    for rec in records:
        # calculate monthly repayment
        r = rec.get('repayment_duration', 0)
        ir_monthly = rec.get('interest_rate', 0) / (12 * 100)
        num = la * ir_monthly * (1 + ir_monthly) ** r
        den = (1 + ir_monthly) ** r - 1
        rec['loan_amount'] = la
        rec['monthly_repayment'] = num / den   # change to Rial
        rec['deposit_amount'] = (la / rec.get('loan_coefficient', 1)) * 100
        calculate_sort_order(rec)

    def valid(rec: Dict[str, Any]) -> bool:
        max_dep = rec.get('maximum_deposit_amount')
        if max_dep and max_dep.lower() != 'nan':
            try:
                if rec.get('deposit_amount') > int(max_dep):
                    return False
            except ValueError:
                pass
        la_lim = rec.get('loan_amount_limit', float('inf'))
        min_la = rec.get('minimum_loan_amount', 0)
        return min_la <= rec.get('loan_amount') <= la_lim
    valid_records = [r for r in records if valid(r)]
    # return valid_records
    return sorted(valid_records, key=lambda x: x.get('sortOrder', 0), reverse=True)

In [7]:
# df_all = pd.DataFrame(_records)
# df_all.head()

In [8]:
# df = pd.DataFrame(update_with_la(_records, la = 700_000_000))
# df.head(15)

In [9]:
# len(update_with_la(_records, la = 700_000_000))

In [10]:
def update_with_da(records: List[Dict[str, Any]], da: float) -> List[Dict[str, Any]]:
    """
    Given deposit amount 'da', update each record's
    loan_amount, monthly_repayment, deposit_amount,
    then filter and return valid records.
    """
    for rec in records:
        coeff = rec.get('loan_coefficient', 0) / 100
        la = coeff * da
        r = rec.get('repayment_duration', 0)
        ir_monthly = rec.get('interest_rate', 0) / (12 * 100)
        num = la * ir_monthly * (1 + ir_monthly) ** r
        den = (1 + ir_monthly) ** r - 1
        rec['loan_amount'] = la
        rec['monthly_repayment'] = num / den  # change to Rial
        rec['deposit_amount'] = da
        calculate_sort_order(rec)

    def valid(rec: Dict[str, Any]) -> bool:
        max_dep = rec.get('maximum_deposit_amount')
        if max_dep and max_dep.lower() != 'nan':
            try:
                if rec.get('deposit_amount') > int(max_dep):
                    return False
            except ValueError:
                pass
        la_lim = rec.get('loan_amount_limit', float('inf'))
        min_la = rec.get('minimum_loan_amount', 0)
        return min_la <= rec.get('loan_amount') <= la_lim

    valid_records = [r for r in records if valid(r)]
    # Sort descending by sortOrder
    return sorted(valid_records, key=lambda x: x.get('sortOrder', 0), reverse=True)



In [11]:
# print(update_with_da(_records, da = 200_000_000))

In [12]:
# len(update_with_da(_records, da = 200_000_000))

In [13]:
def query_complex(
    scenarios: List[Dict[str, Any]],
    deposit_amount: Optional[float] = None,
    repayment_duration: Optional[int] = None,
    deposit_duration: Optional[int] = None,
    interest_rate: Optional[float] = None,
    credit_score: Optional[str] = None
) -> List[Dict[str, Any]]:
    """
    Filter scenarios based on provided parameters.
    """
    matches = []
    for rec in scenarios:
        # deposit range: up to 1.6x
        cond_da = (deposit_amount is None or
                   rec.get('deposit_amount', 0) <= deposit_amount * 1.6)
        cond_rd = (repayment_duration is None or
                   rec.get('repayment_duration') == repayment_duration)
        cond_dep = (deposit_duration is None or
                    rec.get('deposit_duration') == deposit_duration)
        cond_ir = (interest_rate is None or
                   rec.get('interest_rate') == interest_rate)
        cs_field = rec.get('credit_score', '')
        cond_cs = (credit_score is None or
                   (credit_score in cs_field) or
                   (credit_score == 'N' and 'فاقد رتبه' in cs_field))
        if cond_da and cond_rd and cond_dep and cond_ir and cond_cs:
            matches.append(rec)
    return sorted(matches, key=lambda x: x.get('sortOrder', 0), reverse=True)




In [14]:

def get_query_params( _records, 
    deposit__amount: Optional[float] = None,
    repayment__duration: Optional[int] = None,
    deposit__duration: Optional[int] = None,
    interest__rate: Optional[float] = None,
    credit__score: Optional[str] = None,
    loan__amount: Optional[float] = None
) -> List[Dict[str, Any]]:   #Input values are Toman


    if loan__amount:
        scenarios = update_with_la(_records, loan__amount)
    elif deposit__amount and loan__amount is None:
        scenarios = update_with_da(_records, deposit__amount)
    else:
        scenarios = _records

    report = query_complex(
        scenarios,
        deposit_amount=deposit__amount,
        repayment_duration=repayment__duration,
        deposit_duration=deposit__duration,
        interest_rate=interest__rate,
        credit_score=credit__score
    )

    return report

In [15]:
result = get_query_params( _records, 
    loan__amount = 200_000_000,
    deposit__amount= None,
    repayment__duration = None,
    deposit__duration = None,
    interest__rate = 18,
    credit__score = None,
    )

In [16]:
len(result)

30

In [17]:
nickname_0 = result[0].get('nickname', 'نامشخص')
loan_amount_0 = int(result[0].get('loan_amount', 0) / 10_000_000)
monthly_repayment_0 = int(result[0].get('monthly_repayment', 0) // 10)
deposit_amount_0=int(result[0].get('deposit_amount', 0) / 10_000_000)
interest_rate_0=result[0].get('interest_rate', 0)
repayment_duration_0=result[0].get('repayment_duration', 0)
deposit_duration_0=result[0].get('deposit_duration', 0)
credit_score_0=result[0].get('credit_score', 'نامشخص')

In [18]:
result[0]

{'id': 42,
 'nickname': 'شایان',
 'package_name': 'شایان یک',
 'contract_type': 'مرابحه/جعاله',
 'granted_method': 'واریز به حساب',
 'loan_amount_limit': 3000000000,
 'deposit_duration': 3,
 'interest_rate': 18,
 'repayment_duration': 12,
 'loan_coefficient': 80,
 'credit_score': 'B',
 'minimum_deposit_amount': 'nan',
 'maximum_deposit_amount': '15000000000',
 'minimum_loan_amount': 100000000,
 'guarantee': 'بله',
 'receiving_channel': 'غیر حضوری/ حضوری',
 'loan_amount': 200000000,
 'monthly_repayment': 18335998.58124589,
 'deposit_amount': 250000000.0,
 'sortOrder': 0.044667}

In [19]:
msg_filter = f"""{loan_amount_0} میلیون تومان وام {nickname_0} با کارمزد {interest_rate_0} درصد،
 نیاز به {deposit_amount_0}  میلیون تومان سپرده {deposit_duration_0} ماهه،
 با اقساط {monthly_repayment_0}  تومان در {repayment_duration_0} ماه
 با رتبه اعتباری {credit_score_0}.
 """


In [20]:
print(msg_filter)

20 میلیون تومان وام شایان با کارمزد 18 درصد،
 نیاز به 25  میلیون تومان سپرده 3 ماهه،
 با اقساط 1833599  تومان در 12 ماه
 با رتبه اعتباری B.
 


In [21]:
11_612_920

11612920

In [22]:
pd.DataFrame(result).head(10)

,id,nickname,package_name,contract_type,granted_method,loan_amount_limit,deposit_duration,interest_rate,repayment_duration,loan_coefficient,credit_score,minimum_deposit_amount,maximum_deposit_amount,minimum_loan_amount,guarantee,receiving_channel,loan_amount,monthly_repayment,deposit_amount,sortOrder
0,42,شایان,شایان یک,مرابحه/جعاله,واریز به حساب,3000000000,3,18,12,80,B,nan,15000000000,100000000,بله,غیر حضوری/ حضوری,200000000,1.833600e+07,2.500000e+08,0.044667
1,43,شایان,شایان یک,مرابحه,کارت اعتباری,3000000000,3,18,12,100,B,nan,15000000000,100000000,بله,غیر حضوری/ حضوری,200000000,1.833600e+07,2.000000e+08,0.044667
2,10,شایان,شایان یک,مرابحه/جعاله,واریز به حساب,3000000000,3,18,12,80,A,nan,15000000000,100000000,بله,غیر حضوری/ حضوری,200000000,1.833600e+07,2.500000e+08,0.041517
3,11,شایان,شایان یک,مرابحه,کارت اعتباری,3000000000,3,18,12,100,A,nan,15000000000,100000000,بله,غیر حضوری/ حضوری,200000000,1.833600e+07,2.000000e+08,0.041517
4,54,شایان,شایان یک,مرابحه/جعاله,واریز به حساب,3000000000,6,18,12,170,B,nan,15000000000,100000000,بله,غیر حضوری/ حضوری,200000000,1.833600e+07,1.176471e+08,0.040467
5,55,شایان,شایان یک,مرابحه,کارت اعتباری,3000000000,6,18,12,200,B,nan,15000000000,100000000,بله,غیر حضوری/ حضوری,200000000,1.833600e+07,1.000000e+08,0.040467
6,57,شایان,شایان یک,مرابحه/جعاله,واریز به حساب,3000000000,6,18,24,75,B,nan,15000000000,100000000,بله,غیر حضوری/ حضوری,200000000,9.984820e+06,2.666667e+08,0.040467
7,58,شایان,شایان یک,مرابحه,کارت اعتباری,3000000000,6,18,24,70,B,nan,15000000000,100000000,بله,غیر حضوری/ حضوری,200000000,9.984820e+06,2.857143e+08,0.040467
8,71,شایان,شایان یک,مرابحه/جعاله,واریز به حساب,3000000000,3,18,12,70,C,nan,15000000000,100000000,بله,غیر حضوری/ حضوری,200000000,1.833600e+07,2.857143e+08,0.039942
9,72,شایان,شایان یک,مرابحه,کارت اعتباری,3000000000,3,18,12,90,C,nan,15000000000,100000000,بله,غیر حضوری/ حضوری,200000000,1.833600e+07,2.222222e+08,0.039942


# Final version (Deployed V)

In [1]:
import pandas as pd
#------------------------------------
# pd.options.display.max_columns= None
# pd.options.display.max_colwidth= 2
pd.options.display.max_rows = None

#------------------------------------

In [ ]:
import pandas as pd
import re
import json
from typing import List, Dict, Any, Optional

# Load parameter weights once
def _load_weights() -> Dict[str, Any]:
    with open('parameters_weights.json', 'r', encoding='utf-8') as f:
        return json.load(f)

_parameters_weights = _load_weights()



def calculate_sort_order(loan: Dict[str, Any]) -> None:
    
    coef_table = loan.get('loan_coefficient',0)
    
    # Determine bucket
    loanAmountKey = 'out_of_range'
    la = loan.get('loan_amount', 0)
    if la <= 500_000_000:
        loanAmountKey = '1-50'
    elif la <= 1_000_000_000:
        loanAmountKey = '50-100'
    elif la <= 1_500_000_000:
        loanAmountKey = '100-150'
    elif la <= 2_000_000_000:
        loanAmountKey = '150-200'
    elif la <= 2_500_000_000:
        loanAmountKey = '200-250'
    elif la <= 3_000_000_000:
        loanAmountKey = '250-300'

    pw = _parameters_weights
    # Extract individual weights
    ir_key = str(loan.get('interest_rate', ''))
    rd_key = str(loan.get('repayment_duration', ''))
    dd_key = str(loan.get('deposit_duration', ''))
    # CS: first A-E or N
    cs_match = re.search(r'[ABCDE]', loan.get('credit_score', '') or '')
    cs_key = cs_match.group(0) if cs_match else 'N'

    ir_value = pw['IR'][loanAmountKey].get(ir_key, 0)
    rd_value = pw['RD'][loanAmountKey].get(rd_key, 0)
    w_type_coef = pw['w_type'][loanAmountKey].get(loan.get('nickname', ''), 1)
    dd_value = pw['DD'].get(dd_key, 0)
    cs_value = pw['CS'].get(cs_key, 0)

    # Global weights
    w = pw['w']
    coef = (
        ir_value * w['IR_score'] +
        rd_value * w['RD_score'] +
        dd_value * w['DD_score'] +
        cs_value * w['CS_score']
    )
    # Business weight
    w_business = pw['w_business'].get(loan.get('nickname', ''), 1)
    # Calculate final score
    # loan['sortOrder'] = coef * w_type_coef * w_business
    loan['sortOrder'] = coef * w_type_coef * coef_table
    loan['pre_sortOrder'] = coef * w_type_coef 





def update_with_la(records: List[Dict[str, Any]], la: float) -> List[Dict[str, Any]]:
    """
    Given desired loan amount 'la', update each record's
    loan_amount, monthly_repayment, deposit_amount,
    then filter and return valid records.
    """
    for rec in records:
        # calculate monthly repayment
        r = rec.get('repayment_duration', 0)
        ir_monthly = rec.get('interest_rate', 0) / (12 * 100)
        num = la * ir_monthly * (1 + ir_monthly) ** r
        den = (1 + ir_monthly) ** r - 1
        rec['loan_amount'] = la
        rec['monthly_repayment'] = num / den   # change to Rial
        rec['deposit_amount'] = (la / rec.get('loan_coefficient', 1)) * 100
        calculate_sort_order(rec)

    def valid(rec: Dict[str, Any]) -> bool:
        max_dep = rec.get('maximum_deposit_amount')
        if max_dep and max_dep.lower() != 'nan':
            try:
                if rec.get('deposit_amount') > int(max_dep):
                    return False
            except ValueError:
                pass
        la_lim = rec.get('loan_amount_limit', float('inf'))
        min_la = rec.get('minimum_loan_amount', 0)
        return min_la <= rec.get('loan_amount') <= la_lim
    valid_records = [r for r in records if valid(r)]
    # return valid_records
    return sorted(valid_records, key=lambda x: x.get('sortOrder', 0), reverse=True)



def query_complex(
    scenarios: List[Dict[str, Any]],
    deposit_amount: Optional[float] = None,
    repayment_duration: Optional[int] = None,
    deposit_duration: Optional[int] = None,
    interest_rate: Optional[float] = None,
    credit_score: Optional[str] = None
) -> List[Dict[str, Any]]:
    """
    Filter scenarios based on provided parameters.
    """
    matches = []
    for rec in scenarios:
        # deposit range: up to 1.6x
        cond_da = (deposit_amount is None or
                   rec.get('deposit_amount', 0) <= deposit_amount )
        cond_rd = (repayment_duration is None or
                   rec.get('repayment_duration') == repayment_duration)
        cond_dep = (deposit_duration is None or
                    rec.get('deposit_duration') == deposit_duration)
        cond_ir = (interest_rate is None or
                   rec.get('interest_rate') == interest_rate)
        cs_field = rec.get('credit_score', '')
        cond_cs = (credit_score is None or
                   (credit_score in cs_field) or
                   (credit_score == 'N' and 'فاقد رتبه' in cs_field))
        if cond_da and cond_rd and cond_dep and cond_ir and cond_cs:
            matches.append(rec)
    return sorted(matches, key=lambda x: x.get('sortOrder', 0), reverse=True)



## previous

In [10]:
# def update_with_da(records: List[Dict[str, Any]], da: float) -> List[Dict[str, Any]]:
#     """
#     Given deposit amount 'da', update each record's
#     loan_amount, monthly_repayment, deposit_amount,
#     then filter and return valid records.
#     """
#     for rec in records:
#         coeff = rec.get('loan_coefficient', 0) / 100
#         la = coeff * da
#         r = rec.get('repayment_duration', 0)
#         ir_monthly = rec.get('interest_rate', 0) / (12 * 100)
#         num = la * ir_monthly * (1 + ir_monthly) ** r
#         den = (1 + ir_monthly) ** r - 1
#         rec['loan_amount'] = la
#         rec['monthly_repayment'] = num / den  # change to Rial
#         rec['deposit_amount'] = da
#         calculate_sort_order(rec)

#     def valid(rec: Dict[str, Any]) -> bool:
#         max_dep = rec.get('maximum_deposit_amount')
#         if max_dep and max_dep.lower() != 'nan':
#             try:
#                 if rec.get('deposit_amount') > int(max_dep):
#                     return False
#             except ValueError:
#                 pass
#         la_lim = rec.get('loan_amount_limit', float('inf'))
#         min_la = rec.get('minimum_loan_amount', 0)
#         return min_la <= rec.get('loan_amount') <= la_lim

#     valid_records = [r for r in records if valid(r)]
#     # Sort descending by sortOrder
#     return sorted(valid_records, key=lambda x: x.get('sortOrder', 0), reverse=True)

## New

In [24]:
def update_with_da(records: List[Dict[str, Any]], da: float) -> List[Dict[str, Any]]:
 
    for rec in records:
        valid_da = da
       
        if  rec.get('maximum_deposit_amount').lower() != 'nan':
            max_dep = int(rec.get('maximum_deposit_amount'))
            try:
                if  valid_da > max_dep:
                    valid_da = max_dep
            except:
                print("Except")

        coeff = rec.get('loan_coefficient', 0) / 100
        la = coeff * valid_da

        if la > rec.get('loan_amount_limit', float('inf')):
            la = rec.get('loan_amount_limit', float('inf'))
            # coeff = rec.get('loan_coefficient', 0) / 100
            valid_da =  (la / coeff) 

        r = rec.get('repayment_duration', 0)
        ir_monthly = rec.get('interest_rate', 0) / (12 * 100)
        num = la * ir_monthly * (1 + ir_monthly) ** r
        den = (1 + ir_monthly) ** r - 1
        rec['loan_amount'] = la
        rec['monthly_repayment'] = num / den  # change to Rial
        rec['deposit_amount'] = valid_da
        calculate_sort_order(rec)
        # print(f"valid_da :{valid_da}")
        # print(f"la : {la}")

    def valid(rec: Dict[str, Any]) -> bool:
        
        max_dep = rec.get('maximum_deposit_amount')
        if max_dep and max_dep.lower() != 'nan':
            try:
                if rec.get('deposit_amount') > int(max_dep):
                    return False
            except ValueError:
                pass
        la_lim = rec.get('loan_amount_limit', float('inf'))
        min_la = rec.get('minimum_loan_amount', 0)
       
        return min_la <= rec.get('loan_amount') <= la_lim

    valid_records = [r for r in records if valid(r)]
    
    # Sort descending by sortOrder
    return sorted(valid_records, key=lambda x: x.get('sortOrder', 0), reverse=True)



In [25]:
import json
from typing import List, Dict, Any, Optional

# Load parameter weights once
def load_record() -> Dict[str, Any]:
    with open('MEC-LoanRecomn_Scenarios-V17.json', 'r', encoding='utf-8') as f:
        return json.load(f)

_records = load_record()

records_number = len(_records)

def get_query_params( _records, 
    deposit__amount: Optional[float] = None,
    repayment__duration: Optional[int] = None,
    deposit__duration: Optional[int] = None,
    interest__rate: Optional[float] = None,
    credit__score: Optional[str] = None,
    loan__amount: Optional[float] = None
) -> List[Dict[str, Any]]:   #Input values are Toman


    if loan__amount:
        scenarios = update_with_la(_records, loan__amount)
    elif deposit__amount and loan__amount is None:
        scenarios = update_with_da(_records, deposit__amount)
    else:
        scenarios = _records

    report = query_complex(
        scenarios,
        deposit_amount=deposit__amount,
        repayment_duration=repayment__duration,
        deposit_duration=deposit__duration,
        interest_rate=interest__rate,
        credit_score=credit__score
    )

    scenarios_number = len(scenarios)    

    # msg= f"تعداد {len(report)} پیشنهاد وام برای شما پیدا شد.\n\n"
    loan_number = len(report)
    return report, loan_number , records_number, scenarios_number


In [26]:
result , number, records_number , scenarios_number = get_query_params( _records, 
    loan__amount = None,
    deposit__amount= 1_800_000_000,
    repayment__duration = None,
    deposit__duration = None,
    interest__rate = None,
    credit__score = None,
    )

In [27]:
print(number)

150


In [28]:
print(records_number)

159


In [29]:
print(scenarios_number)

150


In [30]:
df_new = pd.DataFrame(result)

In [31]:
df_reduced = df_new.drop(["granted_method","minimum_deposit_amount","minimum_loan_amount","guarantee","receiving_channel","contract_type","package_name"], axis = 1)

In [32]:
df_reduced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      150 non-null    int64  
 1   nickname                150 non-null    object 
 2   loan_amount_limit       150 non-null    int64  
 3   deposit_duration        150 non-null    int64  
 4   interest_rate           150 non-null    int64  
 5   repayment_duration      150 non-null    int64  
 6   loan_coefficient        150 non-null    int64  
 7   credit_score            150 non-null    object 
 8   maximum_deposit_amount  150 non-null    object 
 9   loan_amount             150 non-null    float64
 10  monthly_repayment       150 non-null    float64
 11  deposit_amount          150 non-null    float64
 12  sortOrder               150 non-null    float64
 13  pre_sortOrder           150 non-null    float64
dtypes: float64(5), int64(6), object(3)
memory 

In [33]:
df_reduced

,id,nickname,loan_amount_limit,deposit_duration,interest_rate,repayment_duration,loan_coefficient,credit_score,maximum_deposit_amount,loan_amount,monthly_repayment,deposit_amount,sortOrder,pre_sortOrder
0,2,بهان(بدون ضامن),1000000000,3,23,60,1000,B,nan,1.000000e+09,2.819047e+07,1.000000e+08,101.670000,0.101670
1,1,بهان(بدون ضامن),1000000000,3,23,60,1000,A,nan,1.000000e+09,2.819047e+07,1.000000e+08,97.170000,0.097170
2,101,فرابانک,1500000000,3,23,36,1000,B,nan,1.500000e+09,5.806458e+07,1.500000e+08,63.600000,0.063600
3,45,شایان,3000000000,3,23,12,450,B,15000000000,3.000000e+09,2.822290e+08,6.666667e+08,62.798400,0.139552
4,100,فرابانک,2000000000,3,23,36,1000,A,nan,2.000000e+09,7.741944e+07,2.000000e+08,60.750000,0.060750
5,13,شایان,3000000000,3,23,12,450,A,15000000000,3.000000e+09,2.822290e+08,6.666667e+08,59.490900,0.132202
6,102,فرابانک,1000000000,3,23,36,1000,C,nan,1.000000e+09,3.870972e+07,1.000000e+08,57.312000,0.057312
7,137,نیک‌وام,3000000000,12,4,12,320,B,nan,3.000000e+09,2.554497e+08,9.375000e+08,42.800000,0.133750
8,37,شایان,3000000000,2,23,12,300,B,15000000000,3.000000e+09,2.822290e+08,1.000000e+09,41.865600,0.139552
9,118,نیک‌وام,3000000000,12,4,12,320,A,nan,3.000000e+09,2.554497e+08,9.375000e+08,40.400000,0.126250


In [34]:
df_reduced ["coef_sort"]=df_reduced["loan_coefficient"]*df_reduced["pre_sortOrder"]

In [35]:
df_reduced.head(100)

,id,nickname,loan_amount_limit,deposit_duration,interest_rate,repayment_duration,loan_coefficient,credit_score,maximum_deposit_amount,loan_amount,monthly_repayment,deposit_amount,sortOrder,pre_sortOrder,coef_sort
0,133,نیک‌وام,3000000000,6,4,12,160,B,nan,2.880000e+09,2.452317e+08,1800000000,21.400000,0.133750,21.400000
1,36,شایان,3000000000,2,23,12,150,B,15000000000,2.700000e+09,2.540061e+08,1800000000,20.932800,0.139552,20.932800
2,138,نیک‌وام,3000000000,12,4,24,160,B,nan,2.880000e+09,1.250638e+08,1800000000,20.280000,0.126750,20.280000
3,114,نیک‌وام,3000000000,6,4,12,160,A,nan,2.880000e+09,2.452317e+08,1800000000,20.200000,0.126250,20.200000
4,4,شایان,3000000000,2,23,12,150,A,15000000000,2.700000e+09,2.540061e+08,1800000000,19.830300,0.132202,19.830300
5,152,نیک‌وام,3000000000,6,4,12,160,C,nan,2.880000e+09,2.452317e+08,1800000000,19.600000,0.122500,19.600000
6,119,نیک‌وام,3000000000,12,4,24,160,A,nan,2.880000e+09,1.250638e+08,1800000000,19.080000,0.119250,19.080000
7,157,نیک‌وام,3000000000,12,4,24,160,C,nan,2.880000e+09,1.250638e+08,1800000000,18.480000,0.115500,18.480000
8,59,شایان,3000000000,6,23,24,150,B,15000000000,2.700000e+09,1.414079e+08,1800000000,18.433800,0.122892,18.433800
9,27,شایان,3000000000,6,23,24,150,A,15000000000,2.700000e+09,1.414079e+08,1800000000,17.331300,0.115542,17.331300


In [36]:
df_reduced.sort_values(by="coef_sort", inplace=True, ascending=False)

In [37]:
df_reduced

,id,nickname,loan_amount_limit,deposit_duration,interest_rate,repayment_duration,loan_coefficient,credit_score,maximum_deposit_amount,loan_amount,monthly_repayment,deposit_amount,sortOrder,pre_sortOrder,coef_sort
0,133,نیک‌وام,3000000000,6,4,12,160,B,nan,2.880000e+09,2.452317e+08,1800000000,21.400000,0.133750,21.400000
1,36,شایان,3000000000,2,23,12,150,B,15000000000,2.700000e+09,2.540061e+08,1800000000,20.932800,0.139552,20.932800
2,138,نیک‌وام,3000000000,12,4,24,160,B,nan,2.880000e+09,1.250638e+08,1800000000,20.280000,0.126750,20.280000
3,114,نیک‌وام,3000000000,6,4,12,160,A,nan,2.880000e+09,2.452317e+08,1800000000,20.200000,0.126250,20.200000
4,4,شایان,3000000000,2,23,12,150,A,15000000000,2.700000e+09,2.540061e+08,1800000000,19.830300,0.132202,19.830300
5,152,نیک‌وام,3000000000,6,4,12,160,C,nan,2.880000e+09,2.452317e+08,1800000000,19.600000,0.122500,19.600000
6,119,نیک‌وام,3000000000,12,4,24,160,A,nan,2.880000e+09,1.250638e+08,1800000000,19.080000,0.119250,19.080000
7,157,نیک‌وام,3000000000,12,4,24,160,C,nan,2.880000e+09,1.250638e+08,1800000000,18.480000,0.115500,18.480000
8,59,شایان,3000000000,6,23,24,150,B,15000000000,2.700000e+09,1.414079e+08,1800000000,18.433800,0.122892,18.433800
9,27,شایان,3000000000,6,23,24,150,A,15000000000,2.700000e+09,1.414079e+08,1800000000,17.331300,0.115542,17.331300


In [38]:
# determining the name of the file
file_name = 'TestSorting.xlsx'

# saving the excel
df_reduced.to_excel(file_name)
print('DataFrame is written to Excel File successfully.')

DataFrame is written to Excel File successfully.


In [39]:
# df_reduced.sort_values(by="loan_coefficient", inplace=True, ascending=False)

In [40]:
# df_reduced